In [1]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import pandas as pd
from ultralytics import YOLO
import shutil

In [2]:
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, xmin

In [ ]:
def plot(image_paths, label_paths, num_samples=4):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))
    all_images.sort()
    num_images = len(all_images)
    plt.figure(figsize=(15,12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line [2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w ,h])
                labels.append(label)
            result_image = plot_box(image, bboxes, labels)
            plt.subplot(2, 2, i+1)
            plt.imshow(result_image[:,:,::-1])
            plt.axis('off')
        plt.subplots_adjust(wspace=1)
        plt.tight_layout()
        plt.show()

In [ ]:
%%writefile tumor_v8.yaml
path: '/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection'
train: 'train/images'
val: 'valid/images'
test: 'test/images'

#class names
names:
    0: 'type1'
    1: 'type2'
    2: 'type3'

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
# Assuming `model` is your trained YOLO model and `test_images` is a list of your test images

for image in test_images:
    # Run the model on the test image
    boxes, confidences, class_ids = model.predict(image)

    # Now `boxes` contains the bounding boxes predicted by the model,
    # `confidences` contains the confidence scores for each box, and
    # `class_ids` contains the class IDs for each box.

    # You can compare these predictions to the ground truth to evaluate the model.

# new code from here

In [2]:
#since using only label 1 we need to rewrite the label files
#by removing label 0 and 2 and the scans without any tumor


In [14]:
def clean_data(data_path):
    os.makedirs('newTrain',exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(),'newTrain', 'images'), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), 'newTrain', 'labels'), exist_ok=True)

    labels_path = os.path.join(os.getcwd(), 'train', 'labels')
    img_path = os.path.join(os.getcwd(), 'train', 'images')

    new_labels_path = os.path.join(os.getcwd(), 'newTrain', 'labels')
    new_images_path = os.path.join(os.getcwd(), 'newTrain', 'images')
    
    for file_name in os.listdir(labels_path):
        file_path  = os.path.join(labels_path, file_name)
        with open(file_path, 'r') as f:
            image_file_name = file_name.replace('.txt', '.jpg')
            image_file_path = os.path.join(img_path, image_file_name)
            lines = f.readlines()
            if not lines:
                shutil.copy(file_path, new_labels_path)
                shutil.copy(image_file_path, new_images_path)
            else: 
                lines_id = [line for line in lines if line.startswith('1')]
                if lines_id:
                    with open(os.path.join(new_labels_path, file_name), 'w') as f:
                        f.writelines(lines_id)
                        shutil.copy(image_file_path, new_images_path)
            
    

In [15]:
clean_data('C:/Users/danid/WorkShop/Brain-Tumor-Detection')

In [16]:
#now after the data is cleaned we can start training the model

In [3]:
%%writefile tumor_v8.yaml
path: 'C:/Users/danid/WorkShop/Brain-Tumor-Detection'
train: 'newTrain/images'
val: 'valid/images'
test: 'test/images'

#class names
names:
    0: '1'

Overwriting tumor_v8.yaml


In [4]:
model = YOLO('yolov8m.pt')

In [5]:
EPOCHS = 8
BATCH = 256
IMG_SIZE = 160

In [ ]:
results = model.train(
    data = 'tumor_v8.yaml',
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = BATCH,
    name = 'output'
)

New https://pypi.org/project/ultralytics/8.2.3 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2  Python-3.10.13 torch-2.2.2 CPU (12th Gen Intel Core(TM) i7-12700KF)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=tumor_v8.yaml, epochs=8, time=None, patience=100, batch=256, imgsz=160, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=output4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

train: Scanning C:\Users\danid\WorkShop\Brain-Tumor-Detection\newTrain\labels.cache... 6824 images, 79 backgrounds, 0 corrupt: 100%|██████████| 6824/6824 [00:00<?, ?it/s]
val: Scanning C:\Users\danid\WorkShop\Brain-Tumor-Detection\valid\labels.cache... 1980 images, 17 backgrounds, 1190 corrupt: 100%|██████████| 1980/1980 [00:00<?, ?it/s]

val: WARNING  C:\Users\danid\WorkShop\Brain-Tumor-Detection\valid\images\volume_100_slice_56_jpg.rf.b22da900f5a0e02b66b4854c74d52dbe.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\danid\WorkShop\Brain-Tumor-Detection\valid\images\volume_100_slice_57_jpg.rf.153de57917aeb292aa2e89d1e8fbcde6.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\danid\WorkShop\Brain-Tumor-Detection\valid\images\volume_100_slice_66_jpg.rf.cbc935c59ca711ddc250c6eb6b6cb08b.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\danid\WorkShop\Brain-Tumor-Detection\valid\images\volume_100_slice_75_jpg.rf.a987e0ac1d0a99a62340f5be098550b7.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  C:\Users\danid

Plotting labels to runs\detect\output4\labels.jpg... 
